In [1]:
import numpy as np

In [2]:
a = np.array([[1,2,3],[4,5,6]])

In [3]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [9]:
b = np.array([[2,3,4],[5,6,9]])

In [10]:
b

array([[2, 3, 4],
       [5, 6, 9]])

In [11]:
a

array([[1, 2, 3],
       [4, 5, 6]])

In [12]:
a - b

array([[-1, -1, -1],
       [-1, -1, -3]])

In [13]:
np.argmin([np.linalg.norm(a[i] - b[i]) for i in range(len(a))])

1.7320508075688772

In [14]:
np.min?

In [102]:

"""
This is a module for IDW Spatial Interpolation
"""
import numpy as np
import pandas as pd
from copy import deepcopy
class idw():
    """ A class that is declared for performing IDW Interpolation.
    For more information on how this method works, kindly refer to
    https://en.wikipedia.org/wiki/Inverse_distance_weighting

    Parameters
    ----------
    exponent : positive float, optional
        The rate of fall of values from source data points.
        Higher the exponent, lower is the value when we move
        across space. Default value is 2.
    resolution: str, optional
        Decides the smoothness of the interpolation. Note that
        interpolation is done over a grid. Higher the resolution
        means more grid cells and more time for interpolation.
        Default value is 'standard'
    coordinate_type: str, optional
        Decides the distance metric to be used, while performing
        interpolation. Euclidean by default.     
    """
    def __init__(self, exponent = 2, resolution = 'standard', coordinate_type='Euclidean'):
        
        self.exponent = exponent
        self.resolution = resolution
        self.coordinate_type = coordinate_type
        self.interpolated_values = None
        self.x_grid = None
        self.y_grid = None

    def make_grid(self, x, y, res, offset=0.2):

        """ This function returns the grid to perform interpolation on.
           This function is used inside the fit() attribute of the idw class.
        
        Parameters
        ----------
        x: array-like, shape(n_samples,)
            The first coordinate values of all points where
            ground truth is available
        y: array-like, shape(n_samples,)
            The second coordinate values of all points where
            ground truth is available
        res: int
            The resolution value
        offset: float, optional
            A value between 0 and 0.5 that specifies the extra interpolation to be done
            Default is 0.2
        
        Returns
        -------
        xx : {array-like, 2D}, shape (n_samples, n_samples)
        yy : {array-like, 2D}, shape (n_samples, n_samples)
        """
        y_min = y.min() - offset
        y_max = y.max()+ offset
        x_min = x.min()-offset
        x_max = x.max()+offset
        x_arr = np.linspace(x_min,x_max,res)
        y_arr = np.linspace(y_min,y_max,res)
        xx,yy = np.meshgrid(x_arr,y_arr)  
        return xx,yy

    
    def fit(self, X, y):
        """ The function call to fit the model on the given data. 
        Parameters
        ----------
        X: {array-like, 2D matrix}, shape(n_samples, 2)
            The set of all coordinates, where we have ground truth
            values
        y: array-like, shape(n_samples,)
            The set of all the ground truth values using which
            we perform interpolation

        Returns
        -------
        self : object
            Returns self
        """

#          if self.coordinate_type == 'latlong_small':
# 	 	"""
# 	 		Use the conversions and projections for small changes in LatLong
#  		"""
# 	 	    print ("To be done later")
#             return self

#         if self.coordinate_type == 'latlong_large':
#             """
#                 Code to be written after understanding all the projections.
#             """
#             print ("To be done later")
#             return self

        if self.coordinate_type=="Euclidean":
            
            X = deepcopy(np.c_[X,y])

            if self.resolution=='high':
                xx,yy = self.make_grid(X,y,1000)
                
            if self.resolution=='low':
                xx,yy = self.make_grid(X,y,10)
                
            if self.resolution=='standard':
                xx,yy = self.make_grid(X,y,100)

            new = []
            new_arr = deepcopy(X)
            for points in new_arr:
                min_dist = np.inf
                val = 0
                for j in range(len(yy)):
                    temp = yy[j][0]
                    for i in range(len(xx[0])):
                        dist = np.linalg.norm(np.array([xx[0][i],temp]) - points[:2])
                        if dist<min_dist:
                            min_dist = dist
                            val = (i,j)
                new.append((points,val))
            new_grid = np.zeros((len(xx),len(yy)))
            for i in range(len(new)):
                x = new[i][1][0]
                y = new[i][1][1]
                new_grid[x][y] = new[i][0][2]
            x_nz,y_nz = np.nonzero(new_grid)
            list_nz = []
            for i in range(len(x_nz)):
                list_nz.append((x_nz[i],y_nz[i]))
            final = np.copy(new_grid)
            for i in range(len(xx[0])):
                for j in range(len(yy)):
                    normalise = 0
                    if (i,j) in list_nz:
                        continue
                    else:
                        for elem in range(len(x_nz)):
                            source = np.array([x_nz[elem],y_nz[elem]])
                            target = np.array([xx[0][i],yy[j][0]])
                            dist = (np.abs(xx[0][source[0]] - target[0])**self.exponent + np.abs(yy[source[1]][0] - target[1])**self.exponent)**(1/self.exponent)
                            final[i][j]+=new_grid[x_nz[elem],y_nz[elem]]/dist
                            normalise+=1/(dist)
                    final[i][j]/=normalise
            self.interpolated_values = final
            self.x_grid = xx
            self.y_grid = yy
        
        return self

#     def predict(self, X):
#         """ The function call to predict using the interpolated data
#         Parameters
#         ----------
#         X: {array-like, 2D matrix}, shape(n_samples, 2)
#             The set of all coordinates, where we have ground truth
#             values
        

#         Returns
#         -------
#         y: array-like, shape(n_samples,)
#             The set of all the ground truth values using which
#             we perform interpolation 
#         """
#         if self.coordinate_type == 'Euclidean':
#             for i in range(self.x_grid[0]):
#                 for j in range()
        
#         else:
#             print("Will be done later")
#             return 
            
                
# self.x_grid



In [103]:
a = idw()
import pandas as pd
df = pd.read_csv('test_data/30-03-18.csv')
data = np.array(df[['longitude','latitude','value']])
a.fit(data[:,:2],data[:,2])

In [104]:
a.interpolated_values

array([[171.89189189, 171.89597641, 171.90813547, ..., 173.89050472,
        173.89261459, 173.89466512],
       [171.77142857, 171.77625338, 171.79060316, ..., 173.89585441,
        173.89787202, 173.89983245],
       [171.63636364, 171.64211895, 171.65921778, ..., 173.9012935 ,
        173.90321551, 173.90508269],
       ...,
       [174.49681529, 174.49676176, 174.49660126, ..., 174.24671184,
        174.24416446, 174.24164382],
       [174.49056604, 174.49051451, 174.49035999, ..., 174.24671343,
        174.24419773, 174.2417078 ],
       [174.48447205, 174.48442242, 174.48427358, ..., 174.2466762 ,
        174.24419219, 174.24173298]])